In [5]:
import glob
import os

import pandas as pd
import plotly.express as px
from datasets import load_from_disk

print(os.listdir("Experiments"))
experiment = "Experiments/4_embedding_model"
metrics_files = glob.glob(os.path.join(experiment, "metrics_*.csv"))
metrics_files.sort()
metrics_files

['1_anonymize_parties', '.DS_Store', '4_embedding_model', '2_llm', '3_retrieval_fetch_k']


['Experiments/4_embedding_model/metrics_4a_openai.csv',
 'Experiments/4_embedding_model/metrics_4b_manifestoberta.csv',
 'Experiments/4_embedding_model/metrics_4c_multi-qa-mpnet-base.csv',
 'Experiments/4_embedding_model/metrics_4d_jina-embeddings-v2-base-de.csv',
 'Experiments/4_embedding_model/metrics_4e_mistral_embed.csv',
 'Experiments/4_embedding_model/metrics_4f_e5_base_sts.csv']

In [6]:
# Here we find answers where the RAG chain refused to answer
for file_name in metrics_files:
    df = pd.read_csv(file_name, header=[0, 1], index_col=[0, 1])
    experiment_run = "_".join(os.path.basename(file_name).split("_")[1:]).split(".")[0]
    dataset = load_from_disk(os.path.join(experiment, "dataset_" + experiment_run))

    counter = 0
    exclude_list = []
    parties = list(dataset.keys())
    num_questions = len(dataset[parties[0]])
    for party in parties:
        for q_nr in range(24):
            answer = dataset[party][q_nr]["answer"]
            question = dataset[party][q_nr]["question"]

            if ("keine passende Antwort" in answer) and ("gefunden" in answer):
                counter += 1
                exclude_list.append([question, party])

    print(experiment_run)
    print(f"Fraction of None answers={len(exclude_list)/(len(parties)*num_questions):.3f}")

4a_openai
Fraction of None answers=0.250
4b_manifestoberta
Fraction of None answers=0.306
4c_multi-qa-mpnet-base
Fraction of None answers=0.486
4d_jina-embeddings-v2-base-de
Fraction of None answers=0.326
4e_mistral_embed
Fraction of None answers=0.312
4f_e5_base_sts
Fraction of None answers=0.319


In [7]:
metric_id = 0
df_plot = pd.DataFrame()
for file_name in metrics_files:
    df = pd.read_csv(file_name, header=[0, 1], index_col=[0, 1])
    metrics = list(set(df.columns.get_level_values(0)))
    experiment_run = "_".join(os.path.basename(file_name).split("_")[1:]).split(".")[0]
    df_plot[experiment_run] = df[metrics[metric_id]].mean(axis=0)


df = pd.DataFrame(df_plot).reset_index().rename(columns={"index": "party"})
df_melted = df.melt(id_vars=["party"], var_name="category", value_name="value")
print(metrics[metric_id])
print(df_melted.groupby("category")["value"].mean())
px.bar(
    df_melted,
    x="party",
    y="value",
    color="category",
    barmode="group",
    labels={"value": metrics[metric_id], "party": "Party"},
    title=f"{experiment}",
)

context_relevancy
category
4a_openai                        0.780387
4b_manifestoberta                0.725060
4c_multi-qa-mpnet-base           0.607990
4d_jina-embeddings-v2-base-de    0.736250
4e_mistral_embed                 0.732963
4f_e5_base_sts                   0.692040
Name: value, dtype: float64


In [31]:
metric_id = 1
df_plot = pd.DataFrame()
for file_name in metrics_files:
    df = pd.read_csv(file_name, header=[0, 1], index_col=[0, 1])
    metrics = list(set(df.columns.get_level_values(0)))
    experiment_run = "_".join(os.path.basename(file_name).split("_")[1:]).split(".")[0]
    df_plot[experiment_run] = df[metrics[metric_id]].mean(axis=0)


df = pd.DataFrame(df_plot).reset_index().rename(columns={"index": "party"})
df_melted = df.melt(id_vars=["party"], var_name="category", value_name="value")
print(metrics[metric_id])
print(df_melted.groupby("category")["value"].mean())
px.bar(
    df_melted,
    x="party",
    y="value",
    color="category",
    barmode="group",
    labels={"value": metrics[metric_id], "party": "Party"},
    title=f"{experiment}",
)

IndexError: list index out of range